# Example: image recognition with Keras using CIFAR10 (Work In Progress...)
This notebook explains how to run a simple ConvNN using the Keras with the Cifar10 dataset (Parts of the code come from other's people Kernel in Kaggle)

In [ ]:
#Loading keras deep learnig libraries to build the model: https://keras.io/ 
from keras.datasets import cifar10
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, model_from_json
from keras.callbacks import ReduceLROnPlateau

import matplotlib.pyplot as plt

# Structure of the input dataset
In the CIFAR dataset the images are 32x32x3 (RGB) and they are classified in 10 different categories, namely:

airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

In the next block we will read the dataset:

In [ ]:
# The data, shuffled and split between train and test sets:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
# Input image dimensions
img_rows, img_cols = X_train.shape[1], X_train.shape[2]
# The CIFAR10 images are RGB.
img_channels = X_train.shape[3]

# The CIFAR10 images are 10 different classes.
nb_classes = y_train[1][0]+1

print y_test[5000]
# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
print Y_test[5000]

# ConvNN arquitechture

In [ ]:
#Model Parameters
batch_size = 32 # in each iteration, we consider batch_size training examples at once
num_epochs = 100 # we iterate num_epochs times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
conv_depth_3 = 128 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 256 # the FC layer will have 512 neurons
data_augmentation = True # Whether to use or not data augmentation

#Architecture
inp = Input(shape=(img_rows, img_cols, img_channels)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
#conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1)
conv_2 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(pool_1)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
conv_3 = Convolution2D(conv_depth_3, kernel_size, kernel_size, border_mode='same', activation='relu')(pool_2)
#pool_3 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_3)
#drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
#conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
#conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
#pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
#drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
#flat = Flatten()(drop_2)
flat = Flatten()(conv_3)
hidden = Dense(hidden_size, activation='relu')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
#out = Dense(nb_classes, activation='softmax')(drop_3)
out = Dense(nb_classes, activation='softmax')(hidden)
model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

#print the summary of the architecture
model.summary()

#Visulize the model if desired
from keras.utils.visualize_util import plot
plot(model, to_file='Example_of_CNN_cifar.pdf')

# reduce the learning rate by factor of 0.5 if the validation loss does not get lower in 7 epochs
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=0.0000001, verbose=1)

In [ ]:
# Let's train the model using SGD
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Let's train the SGD model WITHOUT using data augmentation
if not data_augmentation:
    print('Not using data augmentation.')
    history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=num_epochs,
          validation_data=(X_test, Y_test), shuffle=True, callbacks=[reduce_lr])

# Let's train the SGD model using data augmentation
else:
    print('Using real-time data augmentation.')    
  
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False, # apply ZCA whitening
        #Not really needed for MINST because images are centerd, but might work for CIFAR10
        #zoom_range=0.1, 
        rotation_range=20,   
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip = True
    )
    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)
    # Fit the model on the batches generated by datagen.flow().
    history = model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0], #For each epoch generate Xtrain.shape[0] new images for training
                        nb_epoch=num_epochs,
                        validation_data=(X_test, Y_test),
                        #validation_data=(X_train, Y_train),
                        callbacks=[reduce_lr]
                       )


# Learning curves

In [ ]:
# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Accuracy_CIFAR10.pdf')
plt.close()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Loss_CIFAR10.pdf')
plt.close()

# Save the trained model for future usage

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("Example_ConvNN_CIFAR10.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Example_ConvNN_CIFAR10.h5")
print("Saved model to disk")

# Load the model and re-test the performance in test dataset

In [ ]:
# load json and create model
json_file = open("Example_ConvNN_CIFAR10.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Example_ConvNN_CIFAR10.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
score_train = loaded_model.evaluate(X_train, Y_train, verbose=0)
score_test = loaded_model.evaluate(X_test, Y_test, verbose=0)

print "Training %s: %.2f%%" % (loaded_model.metrics_names[1], score_train[1]*100)
print "Test %s: %.2f%%" % (loaded_model.metrics_names[1], score_test[1]*100)